In [1]:
## Steps for running basic GST Algorithms

###     Setup

In [2]:
#Step 1: Import the GST module -- you probably want this at the beginning of every notebook
import pygsti
import json

In [3]:
#Step 2: Load target gateset, dataset, and list of fiducial gate strings. In this case we load directly from files created in past tutorials
gs_target = pygsti.io.load_gateset("tutorial_files/Example_Gateset.txt")
ds = pygsti.io.load_dataset("tutorial_files/Example_Dataset.txt", cache=True)
dsLowCounts = pygsti.io.load_dataset("tutorial_files/Example_Dataset_LowCnts.txt", cache=True)
fiducialList = pygsti.io.load_gatestring_list("tutorial_files/Example_FiducialList.txt")

depol_gateset = gs_target.depolarize(gate_noise=0.1)

#Could also load a fiducial dictionary file like this:
#fiducialDict = GST.load_gatestring_dict("Example_FiducialList.txt")
#fiducialList = fiducialDict.values() #all we really need are the fiducial strings themselves

print "Loaded target gateset with gate labels: ",gs_target.keys()
print "Loaded fiducial list of length: ",len(fiducialList)
print "Loaded dataset of length: ",len(ds)

Loading from cache file:  tutorial_files/Example_Dataset.txt.cache
Loading from cache file:  tutorial_files/Example_Dataset_LowCnts.txt.cache
Loaded target gateset with gate labels:  ['Gi', 'Gx', 'Gy']
Loaded fiducial list of length:  6
Loaded dataset of length:  2737


### LGST to get an initial estimate

In [4]:
#Run LGST to get an initial estimate for the gates in gs_target based on the data in ds
specs = pygsti.construction.build_spam_specs(fiducialGateStrings=fiducialList)
gs_lgst = pygsti.do_lgst(ds, specs, targetGateset=gs_target, svdTruncateTo=4, verbosity=1)

#Gauge optimize the result to match the target gateset
gs_lgst_after_gauge_opt = pygsti.optimize_gauge(gs_lgst, "target", targetGateset=gs_target)

#Contract the result to CPTP
gs_clgst = pygsti.contract(gs_lgst_after_gauge_opt, "CPTP")

In [5]:
print gs_lgst

rhoVec[0] =    0.7096   0.0234  -0.0264   0.7466


EVec[0] =    0.6872  -0.0084   0.0009  -0.6489


Gi = 
   0.9955   0.0094  -0.0005  -0.0030
  -0.0045   0.9273  -0.0507  -0.0053
   0.0160   0.0100   0.8971   0.0237
  -0.0009   0.0026   0.0196   0.9062


Gx = 
   1.0021   0.0017  -0.0021  -0.0004
   0.0097   0.9089  -0.0134  -0.0030
   0.0141  -0.0198  -0.0171   0.9948
  -0.0563  -0.0285  -0.8119   0.0049


Gy = 
   0.9932   0.0041  -0.0154  -0.0084
  -0.0427   0.0114  -0.0031  -0.9818
   0.0087   0.0059   0.8893  -0.0201
  -0.0624   0.8159  -0.0134  -0.0042





### Extended LGST (eLGST)

In [6]:
#Get rho and E specifiers, needed by LGST
specs  = pygsti.construction.build_spam_specs(fiducialGateStrings=fiducialList)
germList = pygsti.io.load_gatestring_list("tutorial_files/Example_GermsList.txt")
maxLengthList = json.load(open("tutorial_files/Example_maxLengths.json","r"))
elgstListOfLists = [ pygsti.io.load_gatestring_list("tutorial_files/Example_eLGSTlist%d.txt" % l) for l in maxLengthList]
                         
gs_exlgst = pygsti.do_iterative_exlgst(ds, gs_clgst, specs, elgstListOfLists, targetGateset=gs_target,
                                  svdTruncateTo=4, verbosity=2)


--- Iterative eLGST: Beginning iter 1 of 10 : 3 gate strings ---
--- eLGST (least squares) ---
  Sum of minimum least squares error (w/out reg terms) = 7.40069e-32
   frobenius distance to target =  0.0573635151888

--- Iterative eLGST: Beginning iter 2 of 10 : 4 gate strings ---
--- eLGST (least squares) ---
  Sum of minimum least squares error (w/out reg terms) = 1.80731e-30
   frobenius distance to target =  0.0573635151888

--- Iterative eLGST: Beginning iter 3 of 10 : 8 gate strings ---
--- eLGST (least squares) ---
  Sum of minimum least squares error (w/out reg terms) = 0.0146807
   frobenius distance to target =  0.0577479001878

--- Iterative eLGST: Beginning iter 4 of 10 : 18 gate strings ---
--- eLGST (least squares) ---
  Sum of minimum least squares error (w/out reg terms) = 0.0793128
   frobenius distance to target =  0.057722007158

--- Iterative eLGST: Beginning iter 5 of 10 : 29 gate strings ---
--- eLGST (least squares) ---
  Sum of minimum least squares error (w/out

### Minimum-$\chi^2$ GST (MC2GST)

In [7]:
#Get lists of gate strings for successive iterations of LSGST to use
specs  = pygsti.construction.build_spam_specs(fiducialGateStrings=fiducialList)
germList = pygsti.io.load_gatestring_list("tutorial_files/Example_GermsList.txt")
maxLengthList = json.load(open("tutorial_files/Example_maxLengths.json","r"))
lsgstListOfLists = [ pygsti.io.load_gatestring_list("tutorial_files/Example_LSGSTlist%d.txt" % l) for l in maxLengthList]
        
gs_mc2 = pygsti.do_iterative_mc2gst(ds, gs_clgst, lsgstListOfLists, verbosity=2,
                                minProbClipForWeighting=1e-6, probClipInterval=(-1e6,1e6) )


--- Iterative LSGST: Beginning iter 1 of 10 : 92 gate strings ---
--- Least Squares GST ---
  Sum of Chi^2 = 48.2156 (92 data params - 40 model params = expected mean of 52; p-value = 0.623488)

--- Iterative LSGST: Beginning iter 2 of 10 : 92 gate strings ---
--- Least Squares GST ---
  Sum of Chi^2 = 48.2156 (92 data params - 40 model params = expected mean of 52; p-value = 0.623489)

--- Iterative LSGST: Beginning iter 3 of 10 : 168 gate strings ---
--- Least Squares GST ---
  Sum of Chi^2 = 130.576 (168 data params - 40 model params = expected mean of 128; p-value = 0.420134)

--- Iterative LSGST: Beginning iter 4 of 10 : 441 gate strings ---
--- Least Squares GST ---
  Sum of Chi^2 = 422.595 (441 data params - 40 model params = expected mean of 401; p-value = 0.219844)

--- Iterative LSGST: Beginning iter 5 of 10 : 817 gate strings ---
--- Least Squares GST ---
  Sum of Chi^2 = 768.626 (817 data params - 40 model params = expected mean of 777; p-value = 0.577799)

--- Iterative L

In [8]:
pygsti.io.write_gateset(gs_exlgst, "tutorial_files/Example_eLGST_Gateset.txt","# Example result from running eLGST")
pygsti.io.write_gateset(gs_mc2,  "tutorial_files/Example_MC2GST_Gateset.txt","# Example result from running MC2GST")

In [9]:
gs_mc2_lowcnts = pygsti.do_iterative_mc2gst(dsLowCounts, gs_clgst, lsgstListOfLists, verbosity=2,
                                        minProbClipForWeighting=1e-6, probClipInterval=(-1e6,1e6) )


--- Iterative LSGST: Beginning iter 1 of 10 : 92 gate strings ---
--- Least Squares GST ---
  Sum of Chi^2 = 71.5877 (92 data params - 40 model params = expected mean of 52; p-value = 0.0371042)

--- Iterative LSGST: Beginning iter 2 of 10 : 92 gate strings ---
--- Least Squares GST ---
  Sum of Chi^2 = 71.5877 (92 data params - 40 model params = expected mean of 52; p-value = 0.0371046)

--- Iterative LSGST: Beginning iter 3 of 10 : 168 gate strings ---
--- Least Squares GST ---
  Sum of Chi^2 = 165.482 (168 data params - 40 model params = expected mean of 128; p-value = 0.0143613)

--- Iterative LSGST: Beginning iter 4 of 10 : 441 gate strings ---
--- Least Squares GST ---
  Sum of Chi^2 = 428.861 (441 data params - 40 model params = expected mean of 401; p-value = 0.162233)

--- Iterative LSGST: Beginning iter 5 of 10 : 817 gate strings ---
--- Least Squares GST ---
  Sum of Chi^2 = 769.85 (817 data params - 40 model params = expected mean of 777; p-value = 0.565529)

--- Iterative

### Maximum Likelihood GST (MLGST)

In [10]:
maxLengthList = json.load(open("tutorial_files/Example_maxLengths.json","r"))
lsgstListOfLists = [ pygsti.io.load_gatestring_list("tutorial_files/Example_LSGSTlist%d.txt" % l) for l in maxLengthList] 

In [11]:
gs_mle = pygsti.do_iterative_mlgst(ds, gs_clgst, lsgstListOfLists, probClipInterval=[-1e2,1e2], verbosity=2)


--- Iterative MLEGST: Beginning iter 1 of 10 : 92 gate strings ---
--- Least Squares GST ---
  Sum of Chi^2 = 48.2156 (92 data params - 40 model params = expected mean of 52; p-value = 0.623489)
    2*Delta(log(L)) = 48.4602

--- Iterative MLEGST: Beginning iter 2 of 10 : 92 gate strings ---
--- Least Squares GST ---
  Sum of Chi^2 = 48.2156 (92 data params - 40 model params = expected mean of 52; p-value = 0.623489)
    2*Delta(log(L)) = 48.4603

--- Iterative MLEGST: Beginning iter 3 of 10 : 168 gate strings ---
--- Least Squares GST ---
  Sum of Chi^2 = 130.576 (168 data params - 40 model params = expected mean of 128; p-value = 0.420139)
    2*Delta(log(L)) = 130.935

--- Iterative MLEGST: Beginning iter 4 of 10 : 441 gate strings ---
--- Least Squares GST ---
  Sum of Chi^2 = 422.578 (441 data params - 40 model params = expected mean of 401; p-value = 0.220013)
    2*Delta(log(L)) = 423.427

--- Iterative MLEGST: Beginning iter 5 of 10 : 817 gate strings ---
--- Least Squares GST

In [12]:
gs_mle_lowcnts = pygsti.do_iterative_mlgst(dsLowCounts, gs_clgst, lsgstListOfLists, 
                                       probClipInterval=[-1e2,1e2], verbosity=2)


--- Iterative MLEGST: Beginning iter 1 of 10 : 92 gate strings ---
--- Least Squares GST ---
  Sum of Chi^2 = 71.5875 (92 data params - 40 model params = expected mean of 52; p-value = 0.0371053)
    2*Delta(log(L)) = 73.4362

--- Iterative MLEGST: Beginning iter 2 of 10 : 92 gate strings ---
--- Least Squares GST ---
  Sum of Chi^2 = 71.5875 (92 data params - 40 model params = expected mean of 52; p-value = 0.0371053)
    2*Delta(log(L)) = 73.4369

--- Iterative MLEGST: Beginning iter 3 of 10 : 168 gate strings ---
--- Least Squares GST ---
  Sum of Chi^2 = 165.482 (168 data params - 40 model params = expected mean of 128; p-value = 0.0143616)
    2*Delta(log(L)) = 169.245

--- Iterative MLEGST: Beginning iter 4 of 10 : 441 gate strings ---
--- Least Squares GST ---
  Sum of Chi^2 = 428.861 (441 data params - 40 model params = expected mean of 401; p-value = 0.162234)
    2*Delta(log(L)) = 440.777

--- Iterative MLEGST: Beginning iter 5 of 10 : 817 gate strings ---
--- Least Squares 

##Compare MLE-GST with LS-GST

In [13]:
gs_mle = pygsti.optimize_gauge(gs_mle,"target",targetGateset=depol_gateset)
gs_mle_lowcnts = pygsti.optimize_gauge(gs_mle_lowcnts,"target",targetGateset=depol_gateset)
gs_mc2 = pygsti.optimize_gauge(gs_mc2,"target",targetGateset=depol_gateset)
gs_mc2_lowcnts = pygsti.optimize_gauge(gs_mc2_lowcnts,"target",targetGateset=depol_gateset)

In [14]:
print "Frobenius diff btwn MLE and datagen = ",gs_mle.frobeniusdist(depol_gateset)
print "Frobenius diff btwn LSGST and datagen = ",gs_mc2.frobeniusdist(depol_gateset)
print "Frobenius diff btwn MLE and LGST = ",gs_mle.frobeniusdist(gs_clgst)
print "Frobenius diff btwn MLE and LSGST = ",gs_mle.frobeniusdist(gs_mc2)
print "Chi^2( LSGST ) = ",pygsti.chi2(ds, gs_mc2, lsgstListOfLists[-1])
print "Chi^2( MLE ) = ",pygsti.chi2(ds, gs_mle, lsgstListOfLists[-1] )
print "LogL( LSGST ) = ",pygsti.logl(gs_mc2, ds, lsgstListOfLists[-1], probClipInterval=[1e-4,1-1e-4])
print "LogL( MLE ) = ",pygsti.logl(gs_mle, ds, lsgstListOfLists[-1], probClipInterval=[1e-4,1-1e-4])

Frobenius diff btwn MLE and datagen =  0.0322203458694
Frobenius diff btwn LSGST and datagen =  0.0356528339106
Frobenius diff btwn MLE and LGST =  0.0119897115646
Frobenius diff btwn MLE and LSGST =  0.00671203679081
Chi^2( LSGST ) =  2799.73646204
Chi^2( MLE ) =  2800.58536739
LogL( LSGST ) =  -4601531.87
LogL( MLE ) =  -4601531.52007


In [15]:
print "Frobenius diff btwn MLE and datagen = ",gs_mle_lowcnts.frobeniusdist(depol_gateset)
print "Frobenius diff btwn LSGST and datagen = ",gs_mc2_lowcnts.frobeniusdist(depol_gateset)
print "Frobenius diff btwn MLE and LGST = ",gs_mle_lowcnts.frobeniusdist(gs_clgst)
print "Frobenius diff btwn MLE and LSGST = ",gs_mle_lowcnts.frobeniusdist(gs_mc2_lowcnts)
print "Chi^2( LSGST ) = ",pygsti.chi2(dsLowCounts, gs_mc2_lowcnts)
print "Chi^2( MLE ) = ",pygsti.chi2(dsLowCounts, gs_mle_lowcnts)
print "LogL( LSGST ) = ",pygsti.logl(gs_mc2_lowcnts, dsLowCounts, probClipInterval=[1e-12,1-1e-12])
print "LogL( MLE ) = ",pygsti.logl(gs_mle_lowcnts, dsLowCounts, probClipInterval=[1e-12,1-1e-12])

Frobenius diff btwn MLE and datagen =  0.0330990970595
Frobenius diff btwn LSGST and datagen =  0.035106972596
Frobenius diff btwn MLE and LGST =  0.0184621381202
Frobenius diff btwn MLE and LSGST =  0.00819647089028
Chi^2( LSGST ) =  2670.40472039
Chi^2( MLE ) =  2684.71978217
LogL( LSGST ) =  -230034.070762
LogL( MLE ) =  -230026.900449


In [16]:
#OLD - construct an old-style gateset, which at one point was needed to keep MLEGST running before
#       it was updated.

#Note: could use gateset with (-1,-1) spam label here to be consistent
#  with trace preservation of MLE (opt_G0 = opt_SP0 = False).
#bUseM1M1Gateset = False
#if bUseM1M1Gateset:
#    gateset_fixTrace = pygsti.construction.build_gateset( [2], [('Q0',)],['Gi','Gx','Gy'], 
#                                 [ "I(Q0)","X(pi/2,Q0)", "Y(pi/2,Q0)"],
#                                 rhoExpressions=["0"], EExpressions=["1"], 
#                                 spamLabelDict={'plus': (0,0), 'minus': (-1,-1) }) #basis="gm" )
#
#    specs = pygsti.construction.build_spam_specs(fiducialGateStrings=fiducialList)
#    gs_lgst_4mle = pygsti.do_lgst(ds, specs, targetGateset=gateset_fixTrace, svdTruncateTo=4, verbosity=1)
#
#    #Gauge optimize the result to match the target gateset
#    gs_lgst_4mle_after_gauge_opt = pygsti.optimize_gauge(gs_lgst_4mle, "target", targetGateset=gs_target)
#
#    #Contract the result to CPTP
#    gs_clgst_4mle = pygsti.contract(gs_lgst_4mle_after_gauge_opt, "CPTP")
#else:
#    gs_clgst_4mle = gs_clgst